# Compute Magnitudes For Standards 

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/10/29
- last update : 2025/10/29 : Generate output file with magnitudes
- last update : 2025-10-30 : find the spectral type

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re


from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
from getCalspec import getCalspec
from getCalspec.getCalspec import getCalspecDataFrame

In [ ]:
from scipy import interpolate

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
machine_name = os.uname().nodename
dm_version = "w_2025_42"
path_rubinsimphot = f"repos/repos_{dm_version}/rubinsimphot/src"
#path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py313")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0

In [ ]:
def CalculateMagnitude(pc,the_sed):
    """
    """
    # compute standard magnitude form the average called std
    mag_std = {}

    atm_bands = pc.bandpass_total_std
    for index,f in enumerate(filter_tagnames) :
        mag_std[f] = the_sed.calc_mag(atm_bands[f])
    return  mag_std

## Imports dedicated to this work

- import the atmospheric transparency emulator (instead of using libradtran code).
- import rubin sim
- import libPhotometricCorrections : encapsulate uninteresting calculation details

### libradtran Emulator

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("LSST")

In [ ]:
WL = emul.GetWL()

#### Library to fit atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

#### Library that encapsulate calculations for Photometric correction

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

In [ ]:
# set default photometric parameters to compute ADU
photoparams = set_photometric_parameters(30, 1 , readnoise=None)

#### library rubin_sim defining LSST parameters, namely for photometric calculations

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

## Configuration

In [ ]:
am0 = 1.20    # airmass
pwv0 = 4.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.0 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

### Initialisation of Atmospheric corrections

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### Check standard atmosphere

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
axs.plot(pc.WL,pc.atm_std,'k-')
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Standard atmosphere transmission")

### Check LSST instrument throughput

Photometric Correction package should find the instrumental passband of LSST

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Instrument throughput (LSST)")

### Check LSST standard Filter throughputs

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total filter throughput (LSST)")

## List of SED 

In [ ]:
ListOfSEDs = ['HD42525' , 'HD111980' , 'HD38666' , 'HD185975' , 'BD-113759' , 'HD 185975',
 'HD 160617' ,  'HD160617' , 'HD142331' , 'HD205905' , 'HD167060' , 'HD2811' ,
 'HD009051' , 'HD14943' , 'HD031128' , 'HD37962' , 'HD38949' , 'HD200654' , 'HD60753', 
 'HD074000' , 'HD115169' , 'HD 38949' , 'HD 38666' , 'HD 42525' , 'HD146233' ,
 'Feige110' , 'HD34816']

## Retrieve getCalspec dataframe

In [ ]:
df = getCalspecDataFrame()
#df = df.set_index("Star_name")
df = df.drop("Unnamed: 0",axis=1)

In [ ]:
df

In [ ]:
# Colonnes où chercher
cols_to_check = ['Star_name','Simbad_Name', 'Alt_Simbad_Name', 'Astroquery_Name', 'Alt_Star_name', 'HD_name']

# Fonction de normalisation (enlève les espaces et met en majuscules)
def normalize(s):
    if pd.isna(s):
        return None
    return str(s).replace(" ", "").upper()

# On crée une copie normalisée
df_norm = df.copy()

# Normalise seulement les colonnes qui existent
for c in cols_to_check:
    if c in df_norm.columns:
        df_norm[c] = df_norm[c].apply(normalize)

# Normalise aussi l'index
df_norm['_index_norm'] = df_norm.index.to_series().apply(normalize)

# Dictionnaire pour stocker les résultats
rows = {}

for name in ListOfSEDs:
    norm_name = normalize(name)
    # Cherche dans toutes les colonnes
    mask = (
        (df_norm['_index_norm'] == norm_name) |
        (df_norm['Star_name'] == norm_name) |
        (df_norm['Simbad_Name'] == norm_name) |
        (df_norm['Alt_Simbad_Name'] == norm_name) |
        (df_norm['Astroquery_Name'] == norm_name) |
        (df_norm['Alt_Star_name'] == norm_name) |
        (df_norm['HD_name'] == norm_name)
    )
    matches = df.loc[mask]
    if not matches.empty:
        # Si plusieurs matches, on garde le premier
        rows[name] = matches.iloc[0]

# Nouveau DataFrame
df_info = pd.DataFrame.from_dict(rows, orient='index')

In [ ]:
df_info

## Retrieve SED spectra from getCalspec

In [ ]:
plt.rcParams["figure.figsize"] = (6,3)

In [ ]:
all_calspec_sed = []
all_calspec_df = []
for idx,sedname in enumerate(ListOfSEDs):
    #print(sedname)
    c = getCalspec.Calspec(sedname)
    the_sed = c.get_spectrum_numpy()
    #c.plot_spectrum()
    all_calspec_sed.append(the_sed)   

##  Convert getCalspec SED in rubin-sim SED

In [ ]:
all_sed_rubinsim = []
dict_sed_rubinsim = {}

for idx,sedname in enumerate(ListOfSEDs):
    #print(sedname)
    the_calspec_sed = all_calspec_sed[idx] 
    the_wavelengths = the_calspec_sed['WAVELENGTH']
    the_flambdas =  the_calspec_sed['FLUX']
    the_wavelengths_arr = the_wavelengths.value/10. # convert in nm
    the_flambdas_arr  = the_flambdas.value*10.  # convert in erg/cm2/s/nm
    # extrapolate these two sed inside the filter range
    if sedname == "BD-113759" or sedname == "HD60753":
        the_wavelengths_arr = np.append(the_wavelengths_arr, 1150.0)
        the_flambdas_arr = np.append(the_flambdas_arr, the_flambdas_arr[-1])
        
    # convert wl in nm and fl in erg/cm2/s/nm
    #mask = ~np.isnan(the_wavelengths_arr) & ~np.isnan(the_flambdas_arr)
    mask = np.isfinite(the_wavelengths_arr) & np.isfinite(the_flambdas_arr)
    the_runbin_sed = Sed(wavelen=the_wavelengths_arr[mask],flambda = the_flambdas_arr[mask] ,name=sedname)
    all_sed_rubinsim.append(the_runbin_sed)
    dict_sed_rubinsim[sedname] = the_runbin_sed

## Compute Magnitudes

In [ ]:
all_ser_magnitudes = []

for idx,sedname in enumerate(ListOfSEDs):

    the_runbin_sed =  all_sed_rubinsim[idx] 
    mags = CalculateMagnitude(pc,the_runbin_sed)
    ser = pd.Series(data=mags,name=sedname)
    all_ser_magnitudes.append(ser)


## Put magnitudes and colors in pandas dataframe

In [ ]:
df_mags = pd.concat(all_ser_magnitudes, axis=1).T  # axis=1 pour les colonnes, puis transpose

In [ ]:
df_mags["u-g"] = df_mags["u"] - df_mags["g"] 
df_mags["g-r"] = df_mags["g"] - df_mags["r"] 
df_mags["r-i"] = df_mags["r"] - df_mags["i"] 
df_mags["i-z"] = df_mags["i"] - df_mags["z"] 
df_mags["z-y"] = df_mags["z"] - df_mags["y"] 

In [ ]:
df_mags.head()

In [ ]:
markdown_table = df_mags.to_markdown(index=True, floatfmt=".3f")

In [ ]:
print(markdown_table)

## Concatenate with info dataframe

In [ ]:
# Fusion par l'index
df_merged = df_mags.join(df_info, how='inner')

In [ ]:
df_mags = df_merged

In [ ]:
df_mags

## Save in pandas dataframe

In [ ]:
df_mags.to_csv("targets_magnitudes.csv") 

In [ ]:
TARGETNAME = "BD-113759"

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    ax1.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    ax1.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    ax1.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
ax1.set_xlabel("$\\lambda$ (nm)")
ax1.set_title("Instrument throughput (LSST)")

ax2 = ax1.twinx()
ax2.plot(dict_sed_rubinsim[TARGETNAME].wavelen, dict_sed_rubinsim[TARGETNAME].flambda,'b-',label=TARGETNAME)
ax2.legend()
plt.show()

In [ ]:
dict_sed_rubinsim["BD-113759"]

In [ ]:
TARGETNAME = "HD60753"  

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    ax1.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    ax1.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    ax1.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
ax1.set_xlabel("$\\lambda$ (nm)")
ax1.set_title("Instrument throughput (LSST)")

ax2 = ax1.twinx()
ax2.plot(dict_sed_rubinsim[TARGETNAME].wavelen, dict_sed_rubinsim[TARGETNAME].flambda,'b-',label=TARGETNAME)
ax2.legend()
plt.show()